### EWAS disease sample

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
sbn.set(style="white",context='notebook')

train=pd.read_csv('C://Users/LSY/Desktop/sample_disease.txt', delimiter=' ')
display(train.head())

unique_categories = train.disease.unique()
display(unique_categories)

### CancerLocator Explore
info:
General cancer 
5mC Methylation 450k microarray/BS-seq
87 Probalistic model for tumor burden
Kang et al. 


In [ ]:
from scipy.stats import beta, describe

train1=pd.read_csv("D:/000Archive/ResearchProjects/DNAm_predictor/CancerLocator-v1.0.1/jasminezhoulab-CancerLocator-e5367a2/example/example_train", delimiter='\t', header=None)
display(train1.head())
test1=pd.read_csv("D:/000Archive/ResearchProjects/DNAm_predictor/CancerLocator-v1.0.1/jasminezhoulab-CancerLocator-e5367a2/example/example_test_methy", delimiter='\t', header=None)
display(test1.head())
cancerstatus = train1[0].unique()
display(cancerstatus)


In [20]:
class CancerPrediction:
    def __init__(self, dens, thetas, cancer_pos):
        n_theta = dens.shape[1]
        n_type = dens.shape[0]
        
        self.theta_preds = np.zeros(n_type)
        self.max_dens = np.zeros(n_type)
        base_dens = dens[:, 0]  # the likelihoods if the sample is normal

        for i in range(n_type):
            row_dens = dens[i, 1:] if cancer_pos else dens[i]
            self.theta_preds[i] = self.theta_pred(row_dens, thetas[1:] if cancer_pos else thetas)
            self.max_dens[i] = np.max(row_dens)

        # sort type by maxDens - baseDens
        self.type_ranks = np.argsort(self.max_dens - base_dens)[::-1]
        self.best_theta = self.theta_preds[self.type_ranks[0]]
        self.best_dens = self.max_dens[self.type_ranks[0]]
        self.best_ratio = self.best_dens - base_dens[self.type_ranks[0]]

    @staticmethod
    def theta_pred(sum_log_dens, thetas):
        theta_index = np.argmax(sum_log_dens)
        if theta_index == -1 or np.max(sum_log_dens) == np.min(sum_log_dens):
            pred = np.nan
        else:
            pred = thetas[theta_index]
        return pred

    def get_best_theta(self):
        return self.best_theta

    def set_best_theta(self, best_theta):
        self.best_theta = best_theta

    def get_theta_preds(self):
        return self.theta_preds

    def set_theta_preds(self, thetas):
        self.theta_preds = thetas

    def get_max_dens(self):
        return self.max_dens

    def set_max_dens(self, max_dens):
        self.max_dens = max_dens

    def get_type_ranks(self):
        return self.type_ranks

    def set_type_ranks(self, type_ranks):
        self.type_ranks = type_ranks

    def get_best_dens(self):
        return self.best_dens

    def set_best_dens(self, best_dens):
        self.best_dens = best_dens

    def get_best_ratio(self):
        return self.best_ratio

    def set_best_ratio(self, best_ratio):
        self.best_ratio = best_ratio


In [65]:
class MethyModel:
    def __init__(self, m):
        self.n_feature = m.shape[1]
        self.n_sample = m.shape[0]
        self.alpha = np.zeros(self.n_feature)
        self.beta = np.zeros(self.n_feature)
        self.mean = np.zeros(self.n_feature)
        self.sd = np.zeros(self.n_feature)
        self.na_ratio = np.zeros(self.n_feature)

        

        for i in range(1,self.n_feature):
            values = m.iloc[:, i]
            display(values)
            non_na = values[~pd.isna(values)]
            non_na_count = len(non_na)

            para = [np.nan, np.nan, np.nan, np.nan]
            if non_na_count >= 2:
                para = est_beta_dist(non_na)

            self.alpha[i] = para[0]
            self.beta[i] = para[1]
            self.mean[i] = para[2]
            self.sd[i] = para[3]
            self.na_ratio[i] = (self.n_sample - non_na_count) / self.n_sample

    def select_feature(self, selected_features):
        new_model = MethyModel()
        selected_indices = np.where(selected_features)[0]

        new_model.alpha = np.copy(self.alpha[selected_indices])
        new_model.beta = np.copy(self.beta[selected_indices])
        new_model.mean = np.copy(self.mean[selected_indices])
        new_model.sd = np.copy(self.sd[selected_indices])
        new_model.na_ratio = np.copy(self.na_ratio[selected_indices])

        new_model.n_feature = len(selected_indices)
        new_model.n_sample = self.n_sample

        return new_model

    def get_n_sample(self):
        return self.n_sample

    def set_n_sample(self, n_sample):
        self.n_sample = n_sample

    def get_n_feature(self):
        return self.n_feature

    def set_n_feature(self, n_feature):
        self.n_feature = n_feature

    def get_alpha(self):
        return self.alpha

    def get_beta(self):
        return self.beta

    def get_mean(self):
        return self.mean

    def get_sd(self):
        return self.sd

    def get_na_ratio(self):
        return self.na_ratio

    def set_alpha(self, alpha):
        self.alpha = alpha

    def set_beta(self, beta):
        self.beta = beta

    def set_mean(self, mean):
        self.mean = mean

    def set_sd(self, sd):
        self.sd = sd

    def set_na_ratio(self, na_ratio):
        self.na_ratio = na_ratio

def est_beta_dist(beta_values):
    mu = np.mean(beta_values)
    var = np.var(beta_values, ddof=1)
    alpha = -mu * (var + mu**2 - mu) / var
    beta_param = (mu - 1) * (var + mu**2 - mu) / var
    return [alpha, beta_param, mu, np.sqrt(var)]

In [22]:
class MethySample:
    def __init__(self, sample_id, sample_type, index, theta, real_data):
        self.id = sample_id
        self.type = sample_type
        self.index = index
        self.theta = theta
        self.real_data = real_data
        self.depth = None
        self.methy = None
        self.dens_ratio = None

    def self_select_feature(self, selected_features):
        
        new_methy = [self.methy[i] for i in range(len(selected_features)) if selected_features[i]]
        
        self.set_methy(new_methy)

    def select_feature(self, selected_features):
        new_sample = MethySample(self.id, self.type, self.index, self.theta, self.real_data)
        
        new_methy = [self.methy[i] for i in range(len(selected_features)) if selected_features[i]]
    
        new_sample.set_methy(new_methy)
        return new_sample

    def get_feature_num(self):
        return len(self.methy)

    def set_type(self, sample_type):
        self.type = sample_type

    def get_type(self):
        return self.type

    def set_index(self, index):
        self.index = index

    def get_index(self):
        return self.index

    def set_theta(self, theta):
        self.theta = theta

    def get_theta(self):
        return self.theta

    def is_real_data(self):
        return self.real_data

    def set_depth(self, depth):
        self.depth = depth

    def get_depth(self):
        return self.depth

    def set_methy(self, methy):
        self.methy = methy

    def get_methy(self):
        return self.methy

    def get_id(self):
        return self.id

    def set_dens_ratio(self, dens_ratio):
        self.dens_ratio = dens_ratio

    def get_dens_ratio(self):
        return self.dens_ratio


In [35]:
class MixModel:
    def __init__(self, tumor, normal, thetas, n_betas, my_threads):
        n_features = tumor.get_na_ratio().shape[0]
        self.n_betas = n_betas
        betas = np.linspace(0, 1, n_betas)
        self.mix_dens = [None] * n_features

        def worker(i):
            tumor_alpha = tumor.get_alpha()[i]
            tumor_beta = tumor.get_beta()[i]
            tumor_dist = beta(tumor_alpha, tumor_beta)

            normal_alpha = normal.get_alpha()[i]
            normal_beta = normal.get_beta()[i]
            normal_dist = beta(normal_alpha, normal_beta)

            mix_dens_i = np.zeros((n_betas, len(thetas)))
            for j, theta in enumerate(thetas):
                beta_dens = np.zeros(n_betas)
                for k, beta_val in enumerate(betas):
                    lower_bound = max(0, (beta_val - 1 + theta) / theta)
                    upper_bound = min(1, beta_val / theta)
                    points = np.linspace(lower_bound, upper_bound, 201)

                    all_tumor_dens = tumor_dist.pdf(points)
                    cal_prob = tumor_dist.cdf(upper_bound) - tumor_dist.cdf(lower_bound)
                    est_prob = simps(all_tumor_dens, points)
                    if est_prob != 0:
                        all_tumor_dens *= cal_prob / est_prob
                    else:
                        all_tumor_dens += 1.0 / len(all_tumor_dens)

                    normal_points = (beta_val - theta * points) / (1 - theta)
                    all_normal_dens = normal_dist.pdf(normal_points[::-1])
                    cal_prob = normal_dist.cdf((beta_val - theta * upper_bound) / (1 - theta)) - \
                               normal_dist.cdf((beta_val - theta * lower_bound) / (1 - theta))
                    est_prob = simps(all_normal_dens[::-1], normal_points)
                    if est_prob != 0:
                        all_normal_dens *= cal_prob / est_prob
                    else:
                        all_normal_dens += 1.0 / len(all_normal_dens)

                    dens = all_tumor_dens * all_normal_dens
                    beta_dens[k] = simps(dens, points)

                norm_term = simps(beta_dens, betas)
                if norm_term != 0:
                    beta_dens /= norm_term
                else:
                    beta_dens += 1.0 / len(beta_dens)

                mix_dens_i[:, j] = beta_dens

            self.mix_dens[i] = mix_dens_i

        with Pool(my_threads) as pool:
            pool.map(worker, range(n_features))

    def select_feature(self, selected_features):
        new_model = MixModel()
        n_selected_feature = np.sum(selected_features)
        new_mix_dens = [self.mix_dens[i] for i in range(len(selected_features)) if selected_features[i]]
        new_model.set_mix_dens(new_mix_dens)
        return new_model

    def get_mix_dens(self):
        return self.mix_dens

    def set_mix_dens(self, mix_dens):
        self.mix_dens = mix_dens

    def get_feature_num(self):
        return len(self.mix_dens)

    def get_theta_num(self):
        return len(self.mix_dens[0][0])

    def get_n_betas(self):
        return self.n_betas

    def set_n_betas(self, n_betas):
        self.n_betas = n_betas

    def __init__(self):
        pass


In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.special import comb

class MethyModel:
    def __init__(self, data_matrix):
        self.data_matrix = data_matrix
        self.alpha = np.zeros(data_matrix.shape[1])
        self.beta = np.zeros(data_matrix.shape[1])

    def get_alpha(self):
        return self.alpha

    def get_beta(self):
        return self.beta

    def select_feature(self, selected_features):
        return self.data_matrix[:, selected_features]

class MixModel:
    def __init__(self, disease_model, ctr_model, thetas_dna, n_betas, n_threads):
        self.disease_model = disease_model
        self.ctr_model = ctr_model
        self.thetas_dna = thetas_dna
        self.n_betas = n_betas
        self.n_threads = n_threads
        self.mix_dens = np.zeros((len(thetas_dna), n_betas, 1))

    def get_mix_dens(self):
        return self.mix_dens

def cal_thetas_dna(thetas, copy_num):
    return thetas

def main():
    # command line args
    config_file = sys.argv[1]

    # read the config file
    prop = {}
    with open(config_file) as config:
        for line in config:
            key, value = line.strip().split('=')
            prop[key] = value

import sys
from scipy.stats import norm
from scipy.special import comb
from concurrent.futures import ThreadPoolExecutor
import time
from decimal import Decimal

class MethyModel:
    def __init__(self, data_matrix):
        self.data_matrix = data_matrix
        self.alpha = np.zeros(data_matrix.shape[1])
        self.beta = np.zeros(data_matrix.shape[1])

    def get_alpha(self):
        return self.alpha

    def get_beta(self):
        return self.beta

    def select_feature(self, selected_features):
        return self.data_matrix[:, selected_features]

class MixModel:
    def __init__(self, disease_model, ctr_model, thetas_dna, n_betas, n_threads):
        self.disease_model = disease_model
        self.ctr_model = ctr_model
        self.thetas_dna = thetas_dna
        self.n_betas = n_betas
        self.n_threads = n_threads
        self.mix_dens = np.zeros((len(thetas_dna), n_betas, 1))

    def get_mix_dens(self):
        return self.mix_dens

def cal_thetas_dna(thetas, copy_num):
    return thetas

def cal_mean(values):
    non_na = [v for v in values if not np.isnan(v)]
    return np.mean(non_na)

def cal_sample_dens(sample, model):
    beta_values = sample.get_methy()
    depths = sample.get_depth()
    mix_dens = model.get_mix_dens()
    n_feature = sample.get_feature_num()
    n_theta = model.get_theta_num()
    n_betas = mix_dens[0].shape[0]
    betas = np.arange(n_betas) / (n_betas - 1.0)
    dens = np.zeros((n_feature, n_theta))

    for i in range(n_feature):
        for j in range(n_theta):
            beta_dens = mix_dens[i][:, j]
            methy_counts = int(round(depths[i] * beta_values[i]))
            dens[i, j] = cal_log_compound_dens(betas, beta_dens, depths[i], methy_counts)

    return dens

def cal_log_compound_dens(betas, beta_dens, n, k):
    log_comb = comb(n, k, exact=True)
    dens = beta_dens * np.power(betas, k) * np.power(1 - betas, n - k)
    prob = integ_simpson(betas, dens)
    log_prob = np.log(prob) if prob != 0 else -1000
    return log_comb + log_prob

def cal_sum_log_dens(theta_dists):
    n_feature, n_theta = theta_dists.shape
    sum_log_dens = np.zeros(n_theta)

    for i in range(n_theta):
        s = 0
        non_na_feature = 0
        for j in range(n_feature):
            log_den = theta_dists[j, i]
            if not np.isnan(log_den):
                non_na_feature += 1
                s += log_den

        if non_na_feature != 0:
            sum_log_dens[i] = s / non_na_feature
        else:
            sum_log_dens[i] = np.nan

    return sum_log_dens

def integ_simpson(points, dens):
    s = 0
    n = len(points) - 1
    h = points[1] - points[0]

    if n == 2:
        s = dens[0] + 4 * dens[1] + dens[2]
    else:
        s = dens[0] + dens[n]
        s += 2 * np.sum(dens[1:n:2])
        s += 4 * np.sum(dens[2:n-1:2])

    s = s * h / 3
    return s

def str_to_int(strings):
    return [0 if s == "NA" else int(s) for s in strings]

def str_to_double(strings):
    return [np.nan if s == "NA" else float(s) for s in strings]

def join(strings, delim):
    return delim.join(strings)

class MethySample:
    def __init__(self, sample_id, depth, methy, features):
        self.sample_id = sample_id
        self.depth = depth
        self.methy = methy
        self.features = features

    def set_depth(self, depth):
        self.depth = depth

    def set_methy(self, methy):
        self.methy = methy

    def self_select_feature(self, selected_features):
        self.features = np.array(selected_features)

    def get_id(self):
        return self.sample_id

    def get_depth(self):
        return self.depth

    def get_methy(self):
        return self.methy

    def get_feature_num(self):
        return len(self.features)

def sample_pred(sub_test_sample, sub_mix_models, disease_types, thetas):
    sample_id = sub_test_sample.get_id()
    n_theta = sub_mix_models[disease_types[0]][2].get_theta_num()

    dens = np.zeros((len(disease_types), n_theta))
    for i, disease_type in enumerate(disease_types):
        type_sub_mix_models = sub_mix_models[disease_type]
        theta_dists = cal_sample_dens(sub_test_sample, type_sub_mix_models[2])
        dens[i, :] = cal_sum_log_dens(theta_dists)

    type_ranks = np.argsort(dens.mean(axis=1))[::-1]
    type_pred = disease_types[type_ranks[0]]
    pred_sample = MethySample(type_pred, thetas[type_ranks[0]])
    pred_sample.set_id(sample_id)
    pred_sample.set_dens_ratio(np.mean(dens[type_ranks[0], :]) / len(sub_test_sample.features))
    return pred_sample

class Predictor:
    def __init__(self, sub_test_sample, sub_mix_models, predictions, disease_types, thetas):
        self.sub_test_sample = sub_test_sample
        self.sub_mix_models = sub_mix_models
        self.predictions = predictions
        self.disease_types = disease_types
        self.thetas = thetas

    def run(self):
        sample_id = self.sub_test_sample.get_id()
        pred_sample = sample_pred(self.sub_test_sample, self.sub_mix_models, self.disease_types, self.thetas)
        self.predictions[sample_id] = pred_sample



In [66]:
MethyModel(train1)

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
495    0.0516
496    0.0326
497    0.0523
498    0.0911
499    0.0660
Name: 1, Length: 500, dtype: float64

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
495    0.1271
496    0.1325
497    0.1122
498    0.1909
499    0.1846
Name: 2, Length: 500, dtype: float64

0         NaN
1      0.9016
2         NaN
3      0.9016
4         NaN
        ...  
495    0.2396
496    0.2519
497    0.2762
498    0.2396
499    0.2592
Name: 3, Length: 500, dtype: float64

0      0.0125
1      0.0000
2      0.0000
3      0.0000
4      0.0000
        ...  
495    0.0432
496    0.0432
497    0.0375
498    0.0323
499    0.0954
Name: 4, Length: 500, dtype: float64

0         NaN
1      0.0000
2      0.0345
3         NaN
4         NaN
        ...  
495    0.0168
496    0.0219
497    0.0165
498    0.0205
499    0.0265
Name: 5, Length: 500, dtype: float64

...

0      0.0035
1      0.0000
2      0.0000
3      0.0000
4      0.0000
        ...  
495    0.0267
496    0.0204
497    0.0209
498    0.0237
499    0.0228
Name: 1996, Length: 500, dtype: float64

0      0.0156
1      0.0156
2         NaN
3      0.0000
4      0.0000
        ...  
495    0.0563
496    0.0518
497    0.0759
498    0.0529
499    0.0519
Name: 1997, Length: 500, dtype: float64

0      0.0506
1      0.0192
2      0.0758
3      0.0000
4      0.1000
        ...  
495    0.1139
496    0.0831
497    0.4379
498    0.1009
499    0.1475
Name: 1998, Length: 500, dtype: float64

0      0.0000
1      0.0167
2      0.0167
3      0.3019
4      0.0705
        ...  
495    0.2436
496    0.4135
497    0.3301
498    0.1630
499    0.3057
Name: 1999, Length: 500, dtype: float64

0      0.0064
1      0.0110
2      0.0110
3      0.0376
4      0.0238
        ...  
495    0.1224
496    0.1236
497    0.0883
498    0.0797
499    0.1438
Name: 2000, Length: 500, dtype: float64